- To rerun any S-LOCAL network you need a minimum of 32GB of RAM/VRAM memory on the device used for training
- Average time to run a 100 epoch on a Nvidia Tesla V100 GPU is 20 minutes
- A faster implementation of the Locally Connected Layer is available in the Keras API : 
- https://keras.io/api/layers/locally_connected_layers/locall_connected2d/

## Imports

In [1]:
INIT_PATH = "."
DOWNLOAD_DATA = False

# To use google colab processing uncomment following lines

#INIT_PATH = "MyDrive/inference_and_learning"
#from google.colab import drive
#MOUNT_PATH = "/content/drive"
#drive.mount(MOUNT_PATH)
#INIT_PATH = MOUNT_PATH + "/" + INIT_PATH
#DOWNLOAD_DATA = True

# To train LOCAL change to True
LOCAL_TRAIN = False 


# To avoid out of memory error, adjust 
NUM_THREADS = 3

In [2]:
import os.path,sys
sys.path.append(INIT_PATH)

import networks as nnets
import importlib
importlib.reload(nnets)
import pandas as pd, torch
from typing import Tuple, List
import torch, torchvision

from time import sleep
import json
from IPython.display import display, HTML
torch.set_default_dtype(torch.float16)

In [3]:
DATA_PATH = "." # Path to data folder
if (DOWNLOAD_DATA):
    torchvision.datasets.CIFAR10(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.CIFAR100(root=DATA_PATH + '/data', download=True)
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True)

    torchvision.datasets.CIFAR10(root=DATA_PATH +'/data', download=True,train = False)
    torchvision.datasets.CIFAR100(root=DATA_PATH +'/data', download=True ,train = False) 
    torchvision.datasets.SVHN(root=DATA_PATH +'/data', download=True, split = "test")


## Training
The following code trains network and save result in files

Two steps are realized :
1. Optimization of the hyperparameters from the optim file
2. Training of the network from the train file
    

In [4]:
trainer = nnets.ThreadedTrainer(
    num_threads= NUM_THREADS,
    DATA_PATH=DATA_PATH,
    SAVE_RESULT_PATH= INIT_PATH + "/files/results.json",
    SAVE_NETWORK_PATH=INIT_PATH + "/networks_saved/",
    SAVE_NON_ZERO = INIT_PATH + "/files/non_zero.json"
)

0 tasks remaining


Accuracy updated every 100 epochs
GPU available: 7.43GB out of 8.35GB
GPU max usage: 0.00GB out of 8.35GB


     Model     |   Optimizer   |    Dataset    ||    optimization_parameters   ||    accuracy   
---------------|---------------|---------------||------------------------------||---------------


Thread    ||  Trainer-0  |  Trainer-1  |  Trainer-2  ||
----------||-------------|-------------|-------------||
Model     || NaN         | NaN         | NaN         ||
Dataset   || NaN         | NaN         | NaN         ||
Optimizer || NaN         | NaN         | NaN         ||
cuda      || cpu         | cpu         | cpu         ||
epoch max || 0           | 0           | 0           ||
Size      || 0           | 0           | 0           ||
----------||-------------|-------------|-------------||
Accuracy  || 0:       0% | 0:       0% | 0:       0% ||


### Optimization


You may use this script to test different hyperparameters
Load the network description in the file files/optim.json
The file is a json file with the following structure :
```json
{
    "optim":{ // Hyperparameters to optimize
        "model":{ // Hyperparameters to optimize for the model
            "S-LOCAL":{ // Model you want to optimize hyperparameters on 
                "hidden_size":[64,128,256,512], // List of values to test
                "dropout":[0.1,0.2,0.3,0.4,0.5] // List of values to test
            }
        }
    },
    "networks":{ // List of networks to test
        "S-LOCAL-64-0.1":{ // Name of the network
            //add the network default description here
        }
    },
    "results":{ // Results of the optimization
        "S-LOCAL-64-0.1":{ //Name of the network
            "best":{ 
                // Best hyperparameters found
            },
            "accuracy":0.9 // Accuracy of the best hyperparameters
        }
    }
}
```

In [5]:

optim = json.load(open(INIT_PATH+"/files/optim.json"))

list_to_do = [key for key,item in optim["networks"].items() if
              key not in optim["results"] and
              not (not LOCAL_TRAIN and item["model"]["name"] in ["S-LOCAL","D-LOCAL"])]

for key in list_to_do:
    nnets.network_thread(optim,key, trainer)
    

### Training

You may use this script to train networks from the /files/train.json file

In [5]:
trainer.acc_update = 100
RETRAIN = False
train_net = json.load(open(INIT_PATH+"/files/train.json"))
for key,item in train_net.items():
    if item["model"]["name"] in ["S-LOCAL","D-LOCAL"] and not LOCAL_TRAIN:
        continue
    if not RETRAIN and os.path.isfile(INIT_PATH+"/networks_saved/"+key+".pth"):
        continue

    trainer.add(key,item,item["model"]["name"] == "S-FC" and item["optimizer"]["name"] == "B-lasso(B=50)")

trainer.logger.big_update()

WARNING !!! :
Trainer-2 error : [Errno 2] No such file or directory: './networks_saved/S-CONV_SGD_CIFAR-100_INCOMPLETE.pth'
Trainer-0 error : [Errno 2] No such file or directory: './networks_saved/S-CONV_SGD_SVHN_INCOMPLETE.pth'


15 tasks remaining
	1. 3-FC_SGD_CIFAR-100                     9. S-FC_B-lasso(B=10)_CIFAR-100
	2. 3-FC_SGD_SVHN                          10. S-FC_B-lasso(B=10)_SVHN
	3. S-FC_SGD_CIFAR-100                     11. S-FC_B-lasso(B=50)_CIFAR-10
	4. S-FC_SGD_SVHN                          12. S-FC_B-lasso(B=50)_CIFAR-100
	5. S-FC_B-lasso(B=0)_CIFAR-10             13. S-FC_B-lasso(B=50)_SVHN
	6. S-FC_B-lasso(B=0)_CIFAR-100            14. S-CONV_SGD_CIFAR-100
	7. S-FC_B-lasso(B=0)_SVHN                 15. S-CONV_SGD_SVHN
	8. S-FC_B-lasso(B=10)_CIFAR-10


Accuracy updated every 100 epochs
GPU available: 5.59GB out of 8.35GB
GPU max usage: 1.49GB out of 8.35GB


     Model     |   Optimizer   |    Dataset    ||    optimization_parameters   ||    accuracy   
------------

## Reproducing the paper


### Table 2

In [ ]:
table2 = pd.read_csv(INIT_PATH+"/files/table2.csv")
results = json.load(open(INIT_PATH+"/files/results.json"))

for dataset in ["CIFAR10","CIFAR100","SVHN"]:
    for line in range(len(table2)):
        iline = table2.iloc[line].to_dict()
        if not iline["train"] : continue
        if iline["dataset"] == "nan": continue
        ID = iline["Model"] + "_" + iline["Training_Method"] + "_" + dataset
        table2.loc[line,dataset] = results[ID]

display(HTML(table2.to_html()))

### figure 3
We can reuse result saved while training table 2

### figure 4